Vamos a entrenar un modelo de lenguaje neuronal feed-forward basado en una ventana de contexto fija y embeddings estáticos. Como datos de entrenamiento, vamos a usar recetas de cocina en español.

-----------------------

Tarea: responder donde dice **PREGUNTA**

## Configuración del entorno

In [ ]:
!pip install -qU datasets spacy watermark

In [ ]:
%%capture
!python -m spacy download es_core_news_sm

In [ ]:
%reload_ext watermark

In [ ]:
%watermark -vmp datasets,spacy,torch,numpy,pandas,tqdm

Para usar GPU, arriba a la derecha seleccionar "Change runtime type" --> "T4 GPU".

Es un buena idea desarrollar con CPU, y usar GPU para la corrida final, para que Google no nos limite el uso. En esta notebook puede ser útil usar GPU.

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

**PREGUNTA 1**: ¿para qué sirve específicamente trabajar con GPU?

## Dataset

Vamos a usar un [corpus de recetas de SomosNLP](https://huggingface.co/datasets/somosnlp/RecetasDeLaAbuela).

In [ ]:
from datasets import load_dataset

dataset = load_dataset("somosnlp/RecetasDeLaAbuela", "version_1")

In [ ]:
# vemos la estructura:
print(dataset)

In [ ]:
# Conservamos pais = "ESP":
dataset = dataset.filter(lambda x: x["Pais"] == "ESP")

In [ ]:
# vemos un ejemplo al azar:
dataset["train"][300]

In [ ]:
# A veces los textos son listas no parseadas como tales.
# En tal caso, hacemos un join de la lista.
import re

def preprocess(example):
    """
    """
    if example["Pasos"].startswith("["):
        pasos_list = eval(example["Pasos"].encode('unicode_escape'))
        example["Pasos"] = " ".join(pasos_list)
    # Eliminamos whitespace duplicado:
    example["Pasos"] = re.sub(r'\s+', ' ', example["Pasos"])
    return example

dataset = dataset.map(preprocess)

In [ ]:
dataset["train"][300]

Hacemos un partición train/test y achicamos (solo para trabajar mas rapido). Y conservamos solo el texto de las recetas.

In [ ]:
dataset = dataset.shuffle(seed=33)

In [ ]:
texts_train = dataset["train"].select(range(0, 4_000))["Pasos"]
texts_test = dataset["train"].select(range(4_000, 8_000))["Pasos"]

In [ ]:
import textwrap

print(textwrap.fill(texts_train[33], 100))

## Construcción del vocabulario y tokenización

Vamos a usar el tokenizer para español de `spacy`.

El objetivo es generar una **lista de n-gramas para entrenar la
NN**. e.g con n=4, queremos tuplas de (3 palabras de contexto, 1 target).

Vamos a:

* Considerar como parte del vocabulario todas las palabras que ocurran al menos dos veces.
* Hacer "padding" con BOS y EOS tokens.
* Tokenizar cada documento y convertir a token IDs según el vocab.
* Pasar de tokens a n-gramas y generar una sola lista con todos los samples de entrenamiento.


In [ ]:
# tokenizer con reglas de puntacion, contracciones, etc:
import spacy

tokenizer = spacy.load('es_core_news_sm')

In [ ]:
# Veamos un ejemplo:
doc = tokenizer(texts_train[0])
print(doc.text)
for i, token in enumerate(doc):
    print(token.text)
    if i > 15:
        break

In [ ]:
from tqdm import tqdm

def create_vocab(docs: list, min_frec=2) -> tuple:
    """Crea un vocabulario a partir de una lista de docs.
    Returns:
        Dos diccionarios: token2idx (palabra -> índice) y idx2token (índice -> palabra)
    """
    # NOTE esto se puede acelerar paralelizando la tokenizacion con datasets.map()
    # y luego usar e.g. pandas explode().value_counts(). Además, podriamos
    # aprovechar y ya guardar el dataset de train tokenizado.
    str2count = {}
    for doc in tqdm(docs):
        for token in tokenizer(doc):
            token = token.text
            str2count[token] = str2count.get(token, 0) + 1
    # filtrar por min_frec:
    str2count = {token: count for token, count in str2count.items() if count >= min_frec}
    # ordenar de mayor a menor frecuencia:
    str2count = dict(sorted(str2count.items(), key=lambda x: x[1], reverse=True))
    # Mapeamos cada token a un índice distinto
    token2idx = {token: idx for idx, token in enumerate(str2count)}
    # Agregamos "<unk>", "<bos>", "<eos>"  al vocab:
    token2idx["<unk>"] = len(str2count)
    token2idx["<bos>"] = len(str2count) + 1
    token2idx["<eos>"] = len(str2count) + 2
    # "Invertir" el diccionario:
    idx2token = {idx: token for idx, token in enumerate(token2idx)}
    return token2idx, idx2token


token2idx, idx2token = create_vocab(texts_train)

In [ ]:
print(len(token2idx))
print(token2idx["<unk>"], token2idx["<bos>"], token2idx["<eos>"])

In [ ]:
from torch import Tensor

def tokenize(doc: str, ngram_order: int = 4) -> Tensor:
  """Convierte documento a tensor de token IDs.
  Agrega n-1 BOS y 1 EOS tokens (end-of-seq. y beg-of-seq).
  """
  token_ids = [token2idx.get(token.text, token2idx["<unk>"]) for token in tokenizer(doc)]
  # agregamos BOS y EOS tokens:
  token_ids = [token2idx["<bos>"]] * (ngram_order - 1) + token_ids + [token2idx["<eos>"]]
  return torch.tensor(token_ids, dtype=torch.long)

print(texts_train[0])
print(tokenize(texts_train[0])[:20])

In [ ]:
def doc2ngrams(doc: str, ngram_order: int = 4) -> list:
  """Convierte un documento en tuplas de
  ([ idx_i-context_size, ..., idx_i-1 ], target_idx), donde cada elemento de la tupla
  es un tensor de token IDs.
  """
  token_ids = tokenize(doc, ngram_order=ngram_order)
  ngrams_list = [
      (token_ids[(i-ngram_order):(i-1)], token_ids[i-1])
      for i in range(ngram_order, len(token_ids) + 1)
  ]
  return ngrams_list

In [ ]:
# por ejemplo:
doc_ = texts_train[0]
token_ids_ = tokenize(doc_)
ngrams_ = doc2ngrams(doc_)

print(doc_)
print(token_ids_[:10])
print(ngrams_)

In [ ]:
# armamos todos los ngrams de training:
ngrams_train = []
for doc in tqdm(texts_train):
  ngrams_train.extend(doc2ngrams(doc, ngram_order=4))

In [ ]:
print(ngrams_train[:2])

## Armado de _batches_

Armamos los batches para entrenar el modelo. Para esto usamos la clase `DataLoader` de PyTorch. En cada iteración, el `DataLoader` nos devuelve un batch de ejemplos. No necesitamos una _collate function_ porque ya todos los ejemplos tienen igual dimensión (no necesitamos padding).

In [ ]:
from torch.utils.data import DataLoader

batch_size = 32

train_loader = DataLoader(ngrams_train, batch_size=batch_size, shuffle=True)

In [ ]:
# Veamos los primeros dos batches de entrenamiento:
torch.manual_seed(33)
for i, data in enumerate(train_loader):
    print(f"### batch {i}")
    print(f"Shapes = {[s.shape for s in data]}")
    print("Primeros 5 ejemplos:")
    print("- Features:")
    print(data[0][:5])
    print("- Targets:")
    print(data[1][:5])
    print()
    if i == 1:
        break

**PREGUNTA 2**: ¿Qué información tiene cada ejemplo en un batch?

**PREGUNTA 3**: ¿para qué sirve hacer procesamiento en batches?

## Modelo

Armamos una red bien sencilla con una hidden layer. Es la misma arquitectura que Figure 7.17 de [Jurafksy](https://web.stanford.edu/~jurafsky/slp3/). Usamos embeddings con inicialización random pero podríamos empezar con embeddings pre-entrenados.

NOTE: Como vamos a usar [Cross Entropy Loss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html), no tenemos que aplicar softmax porque espera "raw, unnormalized scores for each class" i.e. logits.

**PREGUNTA 4**: ¿qué ventaja tiene inicializar la red con embeddings pre-entrenados? ¿qué embeddings podríamos usar para esto?


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class NGramLanguageModel(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_size, ngram_order):
        super().__init__()
        context_size = ngram_order - 1
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, hidden_size)
        self.linear2 = nn.Linear(hidden_size, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs) # shape (bsz, context_size, embed_dim)
        concatenated_embeds = embeds.flatten(1) # shape (bsz, context_size * embed_dim)
        hidden = F.relu(self.linear1(concatenated_embeds)) # shape (bsz, hidden_size)
        logits = self.linear2(hidden) # shape (bsz, vocab_size)
        return logits


**PREGUNTA 5**: ¿en qué atributos de NGramLanguageModel están los pesos de la red?

**PREGUNTA 6**: ¿Qué representa el método forward?

## Entrenamiento


In [ ]:
# Instanciamos el modelo
neural_lm = NGramLanguageModel(
    vocab_size=len(token2idx),
    embedding_dim=50,
    hidden_size=32,
    ngram_order=4,
)
neural_lm = neural_lm.to(device)

In [ ]:
# Funcion de pérdida y optimizador
from torch import optim

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(neural_lm.parameters(), lr=1e-3)

In [ ]:
# Loop de entrenamiento (sin datos de validación)

def train_epoch(model, optimizer, train_loader, log_steps=2000, device=None):
    """Entrena 1 epoch
    """
    total_loss = 0
    steps_done = 0
    n_steps = len(train_loader)
    for context, target in tqdm(train_loader, total=n_steps):
        context = context.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        logits = model(context)
        loss = loss_fn(logits, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        steps_done += 1
        train_loss = total_loss / steps_done
        if steps_done % log_steps == 0:
            print(f"    [steps={steps_done}] train_loss: {train_loss:.4f}")
    return train_loss

def train(
    model, optimizer, train_loader, n_epochs, device=None):
  """Entrena el modelo durante n_epochs.
  """
  for epoch in range(n_epochs):
      print(f"Epoch {epoch} / {n_epochs}")
      epoch_loss = train_epoch(model, optimizer, train_loader, device=device)
      print(f"Training loss = {epoch_loss:.3f}")

**PREGUNTA 7**: ¿Qué es un epoch? ¿Qué es un paso de optimización? ¿Cómo podemos calcular la cantidad máxima de pasos de optimización del entrenamiento?

In [ ]:
# entrenamos!
num_epochs = 1
train(neural_lm, optimizer, train_loader, num_epochs, device=device)

## Generación de texto

In [ ]:
def text2input(context_str: str, ngram_order: int = 4) -> Tensor:
    """Convierte contexto en un input para la NN (tensor de input IDs)
    """
    ngrams = doc2ngrams(context_str, ngram_order=ngram_order)
    # el input es el "contexto" del ultimo ngram
    last_context = ngrams[-1][0]
    # agregamos una dimension que hace las veces de batch (size=1) para hacer el forward
    out = last_context.unsqueeze(0)
    return out

In [ ]:
# Ejemplo:
print(text2input("usamos la", ngram_order=4))
print(text2input("", ngram_order=4))

In [ ]:
def sample_text(model, start_text, max_length=10, ngram_order=4, greedy=False):
    """Generación autorregresiva aleatoria de texto sampleando de softmax.
    El modelo debe ser consistente con ngram_order.
    """
    # buscamos los input IDs segun el context size
    input_ = text2input(start_text, ngram_order=ngram_order)
    # mandamos inputs al mismo device que el modelo
    device = next(model.parameters()).device
    input_ = input_.to(device)
    idx_eos = token2idx["<eos>"]
    context_size = ngram_order - 1
    # el resultado solo va a incluir el contexto usado + el texto nuevo
    idxs_result = input_.clone()
    with torch.inference_mode():
        for i in range(max_length):
            logits = model(input_) # logits
            probas = F.softmax(logits, dim=1) # probas
            if greedy:
                sampled_idx = torch.argmax(probas, dim=1).unsqueeze(1)
            else:
                # sample:
                sampled_idx = torch.multinomial(probas, num_samples=1)
            # actualizamos el resultado
            idxs_result = torch.cat((idxs_result, sampled_idx), dim=1)
            # actualizamos el input conservando solo los ultimos context_size tokens
            input_ = idxs_result[:,-context_size:]
            if sampled_idx == idx_eos:
                break
        tokens_result = [idx2token[idx.item()] for idx in idxs_result.squeeze()]
        return tokens_result

In [ ]:
print(texts_test[0])

In [ ]:
torch.manual_seed(0)
start_text = "1 Cortar los calamares"
res_ = sample_text(neural_lm, start_text, ngram_order=4, max_length=50)

print(" ".join(res_))

In [ ]:
torch.manual_seed(22)
start_text = ""
res_ = sample_text(neural_lm, start_text, ngram_order=4, max_length=50)

print(" ".join(res_))

**PREGUNTA 8**: ¿por qué los textos generados son incoherentes?

In [ ]:
start_text = ""
res_ = sample_text(neural_lm, start_text, ngram_order=4, max_length=50, greedy=True)

print(" ".join(res_))

**PREGUNTA 9**: ¿por qué el texto generado es repetitivo?

## Evaluación

Computamos perplexity (PPL) en test.

* Hacemos $ \exp(\log PPL ) $ para evitar underflow.
* Vean que $\log PPL = CrossEntropy = -avg(\log(probas))$

In [ ]:
ngrams_test = []
for doc in tqdm(texts_test):
  ngrams_test.extend(doc2ngrams(doc, ngram_order=4))

In [ ]:
test_loader = DataLoader(ngrams_test, batch_size=32, shuffle=False)

In [ ]:
def perplexity(model, dataloader, device):
    with torch.no_grad():
        # Iteramos por batch. Vamos a ir guardando las probas de los tokens correctos en cada batch.
        all_log_probs_gt = torch.tensor([], device=device) # gt: ground truth
        for context, target in dataloader:
            context = context.to(device)
            target = target.to(device)
            batch_size = len(target)
            logits = model(context) # shape (bsz, vocab_size)
            log_probs = F.log_softmax(logits, dim=1) # shape (bsz, vocab_size)
            # log_probs_gt:
            log_probs_gt = log_probs[torch.arange(batch_size), target] # shape (bsz)
            all_log_probs_gt = torch.cat((all_log_probs_gt, log_probs_gt))
        # Calculamos PPL:
        ce = -all_log_probs_gt.mean()
        res = torch.exp(ce)
    return res.item()

In [ ]:
test_ppl = perplexity(neural_lm, test_loader, device)
print(f"Test PPL = {test_ppl:.3f}")

**PREGUNTA 10**: ¿El rendimiento de este modelo es mejor o peor que el ngram de la notebook "ngramLM"? ¿Por qué?